# Setup

In [41]:
import pandas as pd
from sklearn.model_selection import train_test_split

# for preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# for model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier


# for error
from sklearn.metrics import mean_absolute_error

# for cross validation score
from sklearn.model_selection import cross_val_score

# for XGBoost
from xgboost import XGBRegressor

# Data

In [2]:
melb_data = pd.read_csv('melb_data.csv')

In [3]:
# Separating target variable and predictors
y = melb_data.Price
X = melb_data.drop(['Price'], axis = 1)

In [4]:
# train and test data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=0)


In [6]:
# selecting low number of unique values in a object column
low_card = [cname for cname in X_train.columns if X_train[cname].nunique()<10
           and X_train[cname].dtype == 'object']

# selecting low number of unique values in a numeric column
num_cols = [cname for cname in X_train.columns 
            if X_train[cname].dtype in ['int64', 'float64']]

# keeping low cardinality and numerical columns only
keep_cols = low_card + num_cols
X_new_train = X_train[keep_cols].copy()
X_new_test = X_test[keep_cols].copy()

In [7]:
X_new_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


# Step 1: Preprocessing

In [9]:
# preprocessing numeric data
num_transformer = SimpleImputer(strategy='constant')

# preprocessing categorical data
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
bundle_preprocessor = ColumnTransformer(transformers=[
    ('num', num_transformer, num_cols),
    ('cat', cat_transformer, low_card)
])

# Step 2: Model 

In [11]:
model = RandomForestRegressor(n_estimators=100, random_state=0)

# Step 3: Create and Evaluate Pipeline

In [13]:
# bundle preprocessing and modeling code in Pipeline
my_pipeline = Pipeline(steps=[
    ('preprocessor', bundle_preprocessor),
    ('model', model)
])

# preprocessing training data, fit model
my_pipeline.fit(X_new_train, y_train)

# preprocessing test data and get predictions
y_pred = my_pipeline.predict(X_new_test)

# Evaluate
score = mean_absolute_error(y_test, y_pred)
print('MAE:', score)

MAE: 160679.18917034855


# Cross Validation


In [14]:
# selecting few columns only as predictors
cols_to_use = ['Rooms', 'Distance', 'Landsize', 'BuildingArea', 'YearBuilt']
X_val = melb_data[cols_to_use]

In [15]:
# defining new pipeline
new_pipeline = Pipeline(steps=[
    ('preprocessor', SimpleImputer()),
    ('model', RandomForestRegressor(n_estimators=50, random_state=0))
])

In [17]:
# Cross validation Score (multiplying by -1 since results are negative)

scores = -1 * cross_val_score(new_pipeline, X_val, y, cv=5, scoring='neg_mean_absolute_error')
print("MAE scores:\n", scores)

MAE scores:
 [301628.7893587  303164.4782723  287298.331666   236061.84754543
 260383.45111427]


In [19]:
# average mean scores
print("Average MAE score")
print(scores.mean())

Average MAE score
277707.3795913405


# XGBoost

In [20]:
#train and test data
XB_train, XB_test, y_train, y_test = train_test_split(X_val, y, test_size=0.2, random_state=0)


In [23]:
# defining the modek
model_XGB = XGBRegressor()
model_XGB.fit(XB_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [24]:
y_pred = model_XGB.predict(XB_test)
print("Mean Absolute Error: " + str(mean_absolute_error(y_pred, y_test)))

Mean Absolute Error: 236021.00835615335


In [25]:
# tuning the parameters
model_XGB = XGBRegressor(n_estimators=500)
model_XGB.fit(XB_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [26]:
# more modification to the model
model_XGB = XGBRegressor(n_estimators=500)
model_XGB.fit(XB_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(XB_test, y_test)],
             verbose=False)

C:\Users\ashok\DemoVirtualEnv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [27]:
# changing learning rate
model_XGB = XGBRegressor(n_estimators=500, learning_rate=0.05)
model_XGB.fit(XB_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(XB_test, y_test)],
             verbose=False)

C:\Users\ashok\DemoVirtualEnv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [28]:
# n_jobs
model_XGB = XGBRegressor(n_estimators=500, learning_rate=0.05, n_jobs=4)
model_XGB.fit(XB_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(XB_test, y_test)],
             verbose=False)

C:\Users\ashok\DemoVirtualEnv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=500, n_jobs=4,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, ...)

In [29]:
y_pred = model_XGB.predict(XB_test)
print("Mean Absolute Error: " + str(mean_absolute_error(y_pred, y_test)))

Mean Absolute Error: 242422.7209131075


# Data Leakage

In [30]:
# for this we will be using different data
credit_data = pd.read_csv('AER_credit_card_data.csv', true_values=['yes'],false_values=['no'])

In [36]:
# target
y_cred = credit_data.card

# predictors
X_cred = credit_data.drop(['card'], axis=1)

# shape
X_cred.shape

(1319, 11)

In [37]:
# exploring data
X_cred.head()

,reports,age,income,share,expenditure,owner,selfemp,dependents,months,majorcards,active
0,0,37.66667,4.5200,0.033270,124.983300,True,False,3,54,1,12
1,0,33.25000,2.4200,0.005217,9.854167,False,False,3,34,1,13
2,0,33.66667,4.5000,0.004156,15.000000,True,False,4,58,1,5
3,0,30.50000,2.5400,0.065214,137.869200,False,False,0,25,1,7
4,0,32.16667,9.7867,0.067051,546.503300,True,False,2,64,1,5


In [42]:
# pipeline
another_pipeline = make_pipeline(RandomForestClassifier(n_estimators=100))

# cv scores
cv_scores = cross_val_score(another_pipeline,X_cred,y_cred, cv=5, scoring='accuracy')

# printing results
print('Cross-validation accuracy: %f'% cv_scores.mean())

Cross-validation accuracy: 0.981052


In [ ]:
# selecting potential leaks
pot_leaks = ['expenditure', 'share', 'active', 'majorcards']
X2_cred = X_cred